In [1]:
import torch
from scripts import mnist, train_utils, architectures

In [2]:
layers = [
    {"n_in": 784, "n_out": 16, "batchnorm": False},
    {"n_out": 32, "batchnorm": True},
    {"n_out": 64, "batchnorm": True},
    {"n_out": 10, "batchnorm": True}
]
net = architectures.MLPCustom(layers)

In [3]:
net

MLPCustom(
  (layers): Sequential(
    (0): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=784, out_features=16, bias=True)
    (2): ReLU()
    (3): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=784, out_features=32, bias=True)
    (5): ReLU()
    (6): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): Linear(in_features=784, out_features=64, bias=True)
    (8): ReLU()
    (9): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Linear(in_features=784, out_features=10, bias=True)
    (11): ReLU()
  )
)